In [1]:
import pandas as pd
import requests
from tqdm import tqdm_notebook
import json
from api_keys import api_key
import os
from sqlalchemy import create_engine
import numpy as np

In [2]:
imdb_data = "movies_metadata.csv"
imdb_data_df = pd.read_csv(imdb_data, low_memory=False)

In [3]:
imdb_id_df = imdb_data_df['imdb_id']

In [4]:
#test = imdb_id_df.head(200)

In [5]:
#test_df = pd.DataFrame(test)

In [6]:
#test_df

In [7]:
url = "http://www.omdbapi.com//?i="
api = f"&apikey={api_key}"

omdb_data = []
for id_ in tqdm_notebook(imdb_id_df):
    response = requests.get(url + id_+api).json()
    director = response['Director']
    actors = response['Actors']
    writers = response['Writer']
    rating = response['Rated']
    awards = response['Awards']
    omdb_data.append({'imdb_id': id_,
                      'director' : director,
                      'actors' : actors,
                      'writers' : writers,
                      'rated' : rating,
                      'awards' : awards})
    #print(response['Director'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
actor_data = []
director_data = []
writer_data = []
rating_data = []
awards_data = []

array_length = len(omdb_data)
for i in range(array_length):
    actor_data.append({'imdb_id': omdb_data[i]['imdb_id'],
                       'actors' : omdb_data[i]['actors'].split(',')})
    director_data.append({'imdb_id': omdb_data[i]['imdb_id'],
                       'directors' : omdb_data[i]['director'].split(',')})
    writer_data.append({'imdb_id': omdb_data[i]['imdb_id'],
                       'writers' : omdb_data[i]['writers'].split(',')})
    rating_data.append({'imdb_id': omdb_data[i]['imdb_id'],
                       'rated' : omdb_data[i]['rated']})
    awards_data.append({'imdb_id': omdb_data[i]['imdb_id'],
                       'awards' : omdb_data[i]['awards']})

        

In [ ]:
actor_data_df = pd.DataFrame(actor_data)
director_data_df = pd.DataFrame(director_data)
writer_data_df = pd.DataFrame(writer_data)
rating_data_df = pd.DataFrame(rating_data)
awards_data_df = pd.DataFrame(awards_data)

In [ ]:
film_actor_df = actor_data_df.set_index(['imdb_id'])['actors'].apply(pd.Series).stack()
film_actor_df = film_actor_df.reset_index()
film_actor_df = film_actor_df.drop('level_1', axis=1)
film_actor_df.columns = ['imdb_id','actor']
film_actor_id_df = film_actor_df
actor_id = film_actor_id_df.groupby('actor').ngroup()
film_actor_id_df['actor_id'] = actor_id
film_actor_id_df['actor_id'] = 'a' + film_actor_df['actor_id'].astype(str)
film_actor_id_df = film_actor_df.drop(['actor'], axis=1)


film_director_df = director_data_df.set_index(['imdb_id'])['directors'].apply(pd.Series).stack()
film_director_df = film_director_df.reset_index()
film_director_df = film_director_df.drop('level_1', axis=1)
film_director_df.columns = ['imdb_id','directors']
film_director_id_df = film_director_df
director_id = film_director_id_df.groupby('directors').ngroup()
film_director_id_df['director_id'] = director_id
film_director_id_df['director_id'] = 'd' + film_director_df['director_id'].astype(str)
film_director_id_df = film_director_df.drop(['directors'], axis=1)

film_writer_df = writer_data_df.set_index(['imdb_id'])['writers'].apply(pd.Series).stack()
film_writer_df = film_writer_df.reset_index()
film_writer_df = film_writer_df.drop('level_1', axis=1)
film_writer_df.columns = ['imdb_id','writers']
film_writer_id_df = film_writer_df
writer_id = film_writer_id_df.groupby('writers').ngroup()
film_writer_id_df['writer_id'] = writer_id
film_writer_id_df['writer_id'] = 'w' + film_writer_df['writer_id'].astype(str)
film_writer_id_df = film_writer_df.drop(['writers'], axis=1)

In [ ]:
actor_df = film_actor_df.drop(['imdb_id'], axis=1)
actor_df.drop_duplicates(subset ="actor_id", keep = 'first', inplace = True) 

for index, row in tqdm_notebook(list(actor_df.iterrows())):
    if len(actor_df.loc[index]['actor'].split()) == 1:
        actor_df.loc[index, 'first_name'] = actor_df.loc[index, 'actor']
        actor_df.loc[index, 'last_name'] = np.nan
    else:
        actor_df.loc[index, 'first_name'] = actor_df.loc[index, 'actor'].rsplit(' ', 1)[0]
        actor_df.loc[index, 'last_name'] = actor_df.loc[index, 'actor'].split()[-1]
actor_df = actor_df.drop(['actor'], axis=1)

In [ ]:
director_df = film_director_df.drop(['imdb_id'], axis=1)
director_df.drop_duplicates(subset ="director_id", keep = 'first', inplace = True) 

for index, row in tqdm_notebook(list(director_df.iterrows())):
    if len(director_df.loc[index]['directors'].split()) == 1:
        director_df.loc[index, 'first_name'] = director_df.loc[index, 'directors']
        director_df.loc[index, 'last_name'] = np.nan
    else:
        director_df.loc[index, 'first_name'] = director_df.loc[index, 'directors'].rsplit(' ', 1)[0]
        director_df.loc[index, 'last_name'] = director_df.loc[index, 'directors'].split()[-1]
director_df = director_df.drop(['directors'], axis=1)

In [ ]:
writer_df = film_writer_df.drop(['imdb_id'], axis=1)
writer_df.drop_duplicates(subset ="writer_id", keep = 'first', inplace = True) 

for index, row in tqdm_notebook(list(writer_df.iterrows())):
    writer_df.loc[index, 'writers'] = writer_df.loc[index]['writers'].split(' (')[0]
    if len(writer_df.loc[index]['writers'].split()) == 1:
        writer_df.loc[index, 'first_name'] = writer_df.loc[index, 'writers']
        writer_df.loc[index, 'last_name'] = np.nan
    else:
        writer_df.loc[index, 'first_name'] = writer_df.loc[index, 'writers'].rsplit(' ', 1)[0]
        writer_df.loc[index, 'last_name'] = writer_df.loc[index, 'writers'].split()[-1]
writer_df = writer_df.drop(['writers'], axis=1)

In [ ]:
rds_connection_string = "postgres:password@localhost:5432/OMDB_Data_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

In [ ]:
test_df.to_sql(name='movie', con=engine, if_exists='append', index=False)
actor_df.to_sql(name='actor', con=engine, if_exists='append', index=False)
director_df.to_sql(name='director', con=engine, if_exists='append', index=False)
writer_df.to_sql(name='writer', con=engine, if_exists='append', index=False)
film_actor_id_df.to_sql(name='film_actor', con=engine, if_exists='append', index=False)
film_director_id_df.to_sql(name='film_director', con=engine, if_exists='append', index=False)
film_writer_id_df.to_sql(name='film_writer', con=engine, if_exists='append', index=False)
#rating_data_df.to_sql(name='movie', con=engine, if_exists='append', index=False)
#awards_data_df.to_sql(name='awards', con=engine, if_exists='append', index=False)